In [52]:
import h2o
import pandas as pd
from h2o.automl import H2OAutoML

In [53]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,1 hour 9 mins
H2O_cluster_timezone:,America/Toronto
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.1
H2O_cluster_version_age:,"21 days, 2 hours and 49 minutes"
H2O_cluster_name:,H2O_from_python_saeed_fd8wf5
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.102 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [54]:
health_insurance = h2o.import_file('data/health_insurance.csv')

Parse progress: |

████████████████████████████████████████████████████████████████| (done) 100%


In [55]:
health_insurance.describe()

Rows:1338
Cols:7

,﻿age,sex,bmi,children,smoker,region,charges
type,int,enum,real,int,enum,enum,real
mins,18.0,,15.96,0.0,,,1121.8739
mean,39.20702541106125,,30.663396860986524,1.0949177877429015,,,13270.422265141255
maxs,64.0,,53.13,5.0,,,63770.42801
sigma,14.049960379216147,,6.098186911679012,1.205492739781914,,,12110.011236693992
zeros,0,,0,574,,,0
missing,0,0,0,0,0,0,0
0,19.0,female,27.9,0.0,yes,southwest,16884.924
1,18.0,male,33.77,1.0,no,southeast,1725.5523
2,28.0,male,33.0,3.0,no,southeast,4449.462


In [56]:
df_train,df_test,df_valid = health_insurance.split_frame(ratios=[.8, .15])

In [58]:
y = "charges"
x = df_train.columns
x.remove(y)

In [59]:
# aml = H2OAutoML(max_models = 25,max_runtime_secs_per_model=30, seed = 623, exclude_algos = ["StackedEnsemble", "DeepLearning"], verbosity="info", nfolds=0)

aml = H2OAutoML(max_models = None, max_runtime_secs = 30, seed = 10, exclude_algos = ["StackedEnsemble", "DeepLearning"], verbosity="info", nfolds=0)


In [60]:
aml.train(x = x, y = y, training_frame = df_train, validation_frame=df_valid)

AutoML progress: |


13:17:22.190: Project: AutoML_9_20231106_131722
13:17:22.190: Cross-validation disabled by user: no fold column nor nfolds > 1.
13:17:22.190: Setting stopping tolerance adaptively based on the training frame: 0.030729254193150105
13:17:22.190: Build control seed: 10
13:17:22.190: training frame: Frame key: AutoML_9_20231106_131722_training_py_41_sid_ae3d    cols: 7    rows: 1059  chunks: 1    size: 16919  checksum: 5524288362087376460
13:17:22.190: validation frame: Frame key: py_43_sid_ae3d    cols: 7    rows: 79  chunks: 1    size: 3933  checksum: 1538186205050039428
13:17:22.190: leaderboard frame: Frame key: py_43_sid_ae3d    cols: 7    rows: 79  chunks: 1    size: 3933  checksum: 1538186205050039428
13:17:22.190: blending frame: NULL
13:17:22.190: response column: charges
13:17:22.190: fold column: null
13:17:22.191: weights column: null
13:17:22.191: Loading execution steps: [{XGBoost : [def_2 (1g, 10w), def_1 (2g, 10w), def_3 (3g, 10w), grid_1 (4g, 90w), lr_search (6g, 30w)]}, 

,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,50.0,50.0,7888.0,3.0,3.0,3.0,7.0,8.0,7.92
,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
,2023-11-06 13:17:46,5.776 sec,0.0,12228.7431689,9223.5443000,149542159.4913590,11156.4127148,8025.0318400,124465544.6621287
,2023-11-06 13:17:46,5.780 sec,5.0,8189.9489356,6334.5491139,67075263.5677393,6951.6404014,5265.1753262,48325304.2705590
,2023-11-06 13:17:46,5.782 sec,10.0,6125.7743576,4632.5659944,37525111.4805265,4657.5396371,3618.1762469,21692675.4709175
,2023-11-06 13:17:46,5.785 sec,15.0,5147.3886126,3616.5127353,26495609.5286591,3448.0952009,2691.0094814,11889360.5144626
,2023-11-06 13:17:46,5.788 sec,20.0,4732.7145470,3061.8055169,22398586.9832667,2945.3594541,2184.2202189,8675142.3139044
,2023-11-06 13:17:46,5.791 sec,25.0,4550.8778589,2768.5462130,20710489.2869176,2780.9639326,1955.2377486,7733760.3945757
,2023-11-06 13:17:46,5.794 sec,30.0,4464.2736163,2631.5843386,19929738.9210038,2721.4966338,1845.0019633,7406543.9279707
,2023-11-06 13:17:46,5.796 sec,35.0,4394.5752842,2530.2029577,19312291.9286948,2676.9937377,1776.8479157,7166295.4716836


In [61]:
lb = aml.leaderboard

In [62]:
lb.head()

model_id,rmse,mse,mae,rmsle,mean_residual_deviance
GBM_grid_1_AutoML_9_20231106_131722_model_49,2712.07,7.35533e+06,1828.59,0.284843,7.35533e+06
XGBoost_grid_1_AutoML_9_20231106_131722_model_48,2792,7.79525e+06,1859.75,0.291538,7.79525e+06
GBM_grid_1_AutoML_9_20231106_131722_model_75,2792.14,7.79607e+06,1803.76,0.282656,7.79607e+06
XGBoost_grid_1_AutoML_9_20231106_131722_model_7,2809.48,7.8932e+06,1874.61,0.282236,7.8932e+06
XGBoost_grid_1_AutoML_9_20231106_131722_model_18,2826.84,7.991e+06,1770.41,0.279016,7.991e+06
XGBoost_grid_1_AutoML_9_20231106_131722_model_38,2828.31,7.99932e+06,1858.04,0.289955,7.99932e+06
GBM_grid_1_AutoML_9_20231106_131722_model_22,2831.35,8.01655e+06,1810.02,0.28168,8.01655e+06
GBM_grid_1_AutoML_9_20231106_131722_model_63,2832.27,8.02173e+06,1866.76,0.279772,8.02173e+06
XGBoost_grid_1_AutoML_9_20231106_131722_model_29,2833.47,8.02853e+06,1837.53,0.284957,8.02853e+06
XGBoost_grid_1_AutoML_9_20231106_131722_model_4,2849.36,8.11882e+06,1882.3,0.291374,8.11882e+06


In [63]:
titanic_pred=aml.leader.predict(df_test)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [64]:
titanic_pred.head()

predict
4490.26
14168.4
12436.4
15933.4
15379.3
4904.49
8025.44
5312.59
14154.1
15083.7


In [65]:
aml.leader.model_performance(df_test)

ModelMetricsRegression: gbm
** Reported on test data. **

MSE: 16967985.00719964
RMSE: 4119.221407887617
MAE: 2400.275932590503
RMSLE: 0.40035108286914406
Mean Residual Deviance: 16967985.00719964